In [1387]:
import pandas as pd
import numpy as np
import math
import re

## Загрузка датасета

In [2]:
ds = pd.read_csv("hillary-clinton-emails/Emails.csv", 
                 delimiter=",", usecols =["MetadataSubject","ExtractedSubject", "ExtractedBodyText","RawText"])

## Предобработка датасета

В качестве текста исходных писем выбиралось поле ExtractedBodyText. 

In [1385]:
ds.ExtractedBodyText[20]

"H <hrod17@clintonernaii.com›\nWednesday, September 12, 2012 11:26 PM\nesullivanjj@state.gov'\nFw: Fwd: more on libya\nLibya 37 sept 12 12,docx\nWe should get this around asap."

На основе "плохих" частотных слов и структуры исходных писем выделила небольшой список стоп-слов и выражений, которые надо удалить из исходного текста перед обработкой. Например, строки с датой и пр.метаинформация.

In [3322]:
stop_words = ["UNCLASSIFIED", "U.S. Department of State"]
stop_lines = ["case no", "doc no", "importance","cc:", "Cc","Sent from", "h <hrod17@clintonemail.com>", 
              "sunday", "monday","tuesday", "wednesday", "thursday","friday", "saturday"]
stop_phrases = ["@state.gov","@clintonemail","©state.gov"]

Удаляю стоп-слова из писем. Слишком короткие письма не включаю в выборку.

In [3323]:
mails = []
titles = []
for i in range(1, len(ds.index)):
    cur_mail = ""
  
    if type(ds.ExtractedBodyText[i]) is str and len(ds.ExtractedBodyText[i]) > 0:
        lines = ds.ExtractedBodyText[i].split("\n")
        cur_mail = ""
        for line in lines:
            for sl in stop_lines:
                if line.lower().startswith(sl):
                    line = ""
                    break
            for sp in stop_phrases:
                if line.find(sp) > -1:
                    line = ""
                    break
            if re.search('.docx', line) != None:
                line = ""
                
            if len(line) != 0:
                cur_word = line.replace("\n", " ")
                for sw in stop_words:
                    cur_word = cur_word.replace(sw, "")
                cur_mail += " " + cur_word + "."
    if len(cur_mail) > 3:
        mails.append(cur_mail)
        if type(ds.ExtractedSubject[i]) is str and len(ds.ExtractedSubject[i]) > 0:
            titles.append(ds.ExtractedSubject[i].replace("\n", " ") +". ")
        else:
            titles.append("")

### Токенизация

На этом этапе удаляю слишком короткие слова(скорее всего они большой смысловой нагрузки не несут) + использую стеммер

In [3324]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [3505]:
stemmer = PorterStemmer()
stemToWord = {}
def tokenize_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if len(token)>3 and re.match('[A-Za-z]+', token) != None:
            filtered_tokens.append(token.strip("."))
    stems = []
    tags = nltk.pos_tag(filtered_tokens)
    for token, tag in tags:
        stem = stemmer.stem(token)
        if len(stem)>3 and (tag[0] == "N" or tag[0]=="V"):
            stems.append(stem)
            stemToWord[stem]=token
    return stems

In [3506]:
tokenize_and_stem(mails[0])

['latest', 'syria', 'qaddafi', 'march', 'hillari']

In [3507]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict

def filterFrequentWords(n, percent, mails):
    vectorizer = CountVectorizer(ngram_range=(n,n),tokenizer=tokenize_and_stem, stop_words=stopwords.words("english"))
    X = vectorizer.fit_transform(mails)
    
    ngramms = defaultdict(int)
    END_OF_SENTENCE = "<END>"
    feature_names = vectorizer.get_feature_names()
    count = [len(np.nonzero(X[0,:])[1]) ]
    wordsCount = 0
    for i in X.nonzero()[1]:
        ngramms[feature_names[i]] += 1
        wordsCount+=1
    
    countToToken = defaultdict(list)
    for key,value in ngramms.items():
        countToToken[value].append(key)
    maxCount = sorted(list(countToToken), reverse=True)
    frequentTokens = [] 
    for c in maxCount:
        frequentTokens += countToToken[c] 
    return frequentTokens[0:int(wordsCount*percent)]
    

## Биграммы

In [3508]:
BiGramms = filterFrequentWords(2, 1, mails)

## Коллокации

In [3509]:
from nltk.collocations import *

In [3510]:
from nltk.tokenize import word_tokenize, sent_tokenize, TweetTokenizer
words = []
END_OF_SENTENCE = "<END>"

for mail in mails:
    words.extend(word_tokenize(mail))
    words.append(END_OF_SENTENCE)

In [3511]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(words)
finder.nbest(bigram_measures.pmi, 10) 

[("'1.com", 'aeirig'),
 ("'1HE", ".i't"),
 ("'Banish", 'Sexual'),
 ("'H", 'CCL'),
 ("'Li", 'Matou'),
 ("'Stone", 'Harbour'),
 ("'bad", 'talkers'),
 ("'civ", 'cas'),
 ("'ffrjend", 'nieei'),
 ("'i", 'Voz')]

In [3512]:
def countNGramms(n, mails, indexes):
    model = defaultdict(int)
    
    END_OF_SENTENCE = "<END>"
    for id in indexes:
        words = tokenize_and_stem(mails[id])
            
        for i in range(n-1, len(words)):
            ngramma = "_".join(words[i - (n-1):i+1])
            model[ngramma] += 1
    countToModel = defaultdict(list)
    for key, value in model.items():
        countToModel[value].append(key)
    sortedCount = sorted(countToModel.keys(),reverse=True)
    print([countToModel[c] for c in sortedCount[:5]])

## Обучение

Ищу потенциальные стоп-слова - слова, которые встречаются в большом количестве документов(высокая df). Параметром max_df не пользовалась, потому что хотелось посмотреть на частотные слова.

In [3513]:
stopUniGramms = filterFrequentWords(1, 0.0003, mails)


In [3514]:
print(len(stopUniGramms))
print(stopUniGramms)

37
['call', 'work', 'state', 'time', 'want', 'know', 'meet', 'talk', 'secretari', 'today', 'think', 'need', 'make', 'offic', 'tomorrow', 'presid', 'come', 'issu', 'thank', 'take', 'said', 'week', 'year', 'discuss', 'hous', 'peopl', 'help', 'part', 'secur', 'report', 'sent', 'includ', 'govern', 'follow', 'point', 'support', 'hope']


In [3656]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=50000,ngram_range=(1,2),tokenizer=tokenize_and_stem, 
                             stop_words=stopwords.words("english")+stopUniGramms)
X = vectorizer.fit_transform(mails)

In [3657]:

feature_names = vectorizer.get_feature_names()

In [3658]:
len(feature_names)

50000

In [3662]:
n_clusters=7
from sklearn.cluster import KMeans
km = KMeans(n_clusters=n_clusters)


In [3663]:
km.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=7, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

## Результаты

Количество писем для каждого кластера. При любых экспериментах выделяется крупный кластер и несколько маленьких. 
Маленькие кластеры характеризуются редкими словами. Большой описывается более частотными.

In [3664]:
km.labels_.sort()
curIndex = 0
curCount = 0
for i in range(len(km.labels_)):
    if km.labels_[i] != curIndex:
        print(curIndex, curCount)
        curCount = 0
        curIndex += 1

    curCount += 1
print(curIndex, curCount)
    

0 125
1 5420
2 79
3 103
4 219
5 79
6 690


Нашла примеры использования описания центра кластера в качестве характеристики. 
Кажется, что поиски самых частотных  n-грамм не очень хорошо работают.

In [3666]:

order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(n_clusters):
    print("Cluster ", i, " centroid")
    print(", ".join([
            " ".join([stemToWord[name] for name in feature_names[ind].split(" ")])
                for ind in order_centroids[i, :10]]))
    print()
    print() 
    
  

Cluster  0  centroid
department, room, route, residence, department department, conference room, conference, benghazi, redactions, agreement information


Cluster  1  centroid
released, email, check, schedule, message, morning, list, hillary, looks, copy


Cluster  2  centroid
prepare, signed, prepare signed, letter, thing, review, morning, prepare statement, statement, monday


Cluster  3  centroid
print, print copy, copy, leverage, fine, types, lauren, influence, haiti, militancy create


Cluster  4  centroid
sends, draft, done, speech, edited, sheet, review, revisions, letter, copy


Cluster  5  centroid
confirm, oscar, morning, confirm fact, confirm morning, sheet, fact, check, confirm connection, espinosa


Cluster  6  centroid
huma, assistant, cheryl, lona, abedin, clinton, valmoro, mills, policy, valmoro assistant




## Оценка

1) Цель: проверить, насколько хорошо связаны слова внутри кластеров

Задание: найдите лишнее слово.

Предлагается 5 слов, описывающих кластер, + слово из другого кластера
Для каждого кластера, получаем величину 1-"Кол-во ошибок/кол-во асессоров"

UPD: Опрос показал, что лучше искать несколько лишних слов.



Проблема с интрепретацией самого большого кластера. 

2) Цель: проверить, насколько хорошо различаются слова из разных кластеров

Задание:  Даны n+1 слов из n  кластеров(при этом 2 слова из одного кластера). Найдите пару.

Для каждого кластера, получаем величину 1- "Кол-во ошибок/кол-во асессоров"

UPD: Лучше выбирать небольшое количество слов. Здесь стало показательным, что кластеризация плохая.


3) Цель: проверить, насколько хорошо связаны слова внутри кластеров

Задание: Дано описание кластера. Исключите лишние слова.


Итог: С 3 заданием сработало лучше всего. В целом кластеризация не очень удалась. 
Очертания тем выделяются.